# Horse tracking

Data provided by Prof. Lene Fischer

In [ ]:
%matplotlib inline

In [ ]:
import urllib
import os
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame, read_file

import sys
sys.path.append("..")
import movingpandas as mpd

import warnings
warnings.simplefilter("ignore")

In [ ]:
df = read_file('data/demodata_horse_collar.gpkg')
df['t'] = pd.to_datetime(df['timestamp'])
df = df.set_index('t').tz_localize(None)
print("Finished reading {} rows".format(len(df)))
df.head()

In [ ]:
tc = mpd.TrajectoryCollection(df, 'CollarID')
print(tc)

In [ ]:
tc.plot()

In [ ]:
daily = tc.split_by_date(mode='day')

In [ ]:
daily.trajectories[1].plot(column='speed', legend=True, cmap='RdYlGn')

In [ ]:
daily.trajectories[1].hvplot(width=500, height=500, line_width=5, c='speed', cmap='RdYlGn') 

In [ ]:
size=350
(daily.trajectories[2].hvplot(width=size, height=size, line_width=5, c='speed', cmap='RdYlGn') +
daily.trajectories[3].hvplot(width=size, height=size, line_width=5, c='speed', cmap='RdYlGn', tiles='CartoEco') +
daily.trajectories[4].hvplot(width=size, height=size, line_width=5, c='speed', cmap='RdYlGn', tiles='StamenTerrain'))